<a href="https://colab.research.google.com/github/amanichivilkar/Bike-Sharing-Demand-Prediction/blob/main/Amani_Chivilkar_Bike_Sharing_Demand_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b> Project Title : Seoul Bike Sharing Demand Prediction </b>
----
### Currently Rental bikes are introduced in many urban cities for the enhancement of mobility comfort. It is important to make the rental bike available and accessible to the public at the right time as it lessens the waiting time. Eventually, providing the city with a stable supply of rental bikes becomes a major concern. The crucial part is the prediction of bike count required at each hour for the stable supply of rental bikes.


### **Import of Python Libraries and Defining Function**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns 
%matplotlib inline
sns.set_theme(style="whitegrid")
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

#Scoring
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# ML
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor  
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/data/SeoulBikeData.csv", encoding ="ISO-8859-1")

In [ ]:
%pip install mlxtend --upgrade

     |████████████████████████████████| 1.3 MB 5.3 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
!pip3 install lime

     |████████████████████████████████| 275 kB 5.2 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=eb8adabe9a5c60539b3dbce241b16a8ec6af9da290776feeb1a0dc3d4eeb885e
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [ ]:
import lime
import lime.lime_tabular

In [ ]:
pip install shap

     |████████████████████████████████| 564 kB 5.3 MB/s 


In [ ]:
import shap

In [ ]:
# This gives a dataframe with lable Encoder Applied on the categorical featuure
from sklearn.preprocessing import LabelEncoder
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
# PLOT weight of the features for decision tree and random forest
def weight(ml):  
     ml.fit(x_train,y_train)
     y_pred = ml.predict(x_test)

     # plot
     df5=pd.DataFrame({"feature":independent,"weight":ml.feature_importances_}).sort_values('weight',ascending=False).head(10)

     plt.figure(figsize=(10,5))
    
     sns.barplot(x=df5.weight,y=df5.feature).set(title=f'Top 10 Important features for {ml}')
     

In [ ]:
# Get the LIME explanation by giving ml modle and sample number to the funtion "explanation"

def explanation(ml,obs_number):
    ml.fit(x_train,y_train)
    y_pred = ml.predict(x_test)

    explanation = explainer.explain_instance(x_test.values[obs_number], ml.predict, num_features=14)
    print(f'predicted_value={y_pred[obs_number]}')
    print(f'Actual_value={y_test.values[obs_number]}')
    explanation.show_in_notebook(show_all=False)

### **Preprosessing Data**

<b> Data Description </b>

<b> The dataset contains weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour and date information.</b>


*  Date : year-month-day
*  Rented Bike count - Count of bikes rented at each hour
*  Hour - Hour of the day
*  Temperature-Temperature in Celsius
*  Humidity - %
*  Windspeed - m/s
*  Visibility - 10m
*  Dew point temperature - Celsius
*  Solar radiation - MJ/m2
*  Rainfall - mm
*  Snowfall - cm
*  Seasons - Winter, Spring, Summer, Autumn
*  Holiday - Holiday/No holiday
*  Functional Day - NoFunc(Non Functional Hours), Fun(Functional hours)

In [ ]:
df.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(°C)            8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(°C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

In [ ]:
# Convert date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Create Month, Weekdays and Year columns
df['month']=df['Date'].apply(lambda x: x.month)
df['weekday']=df['Date'].dt.day_name()
df['year']=df['Date'].apply(lambda x: x.year)

In [ ]:
# Rename columns
df.rename(columns={'Date':"date", 'Rented Bike Count':'rented_bike_count', 'Hour':"hour", 'Temperature(°C)':'temp', 'Humidity(%)':'humidity%',
       'Wind speed (m/s)':'wind_speed', 'Visibility (10m)':'visibility', 'Dew point temperature(°C)':'dev_point_temp',
       'Solar Radiation (MJ/m2)':'solar_radiation', 'Rainfall(mm)':'rainfall', 'Snowfall (cm)':'snowfall', 'Seasons':'season',
       'Holiday':'holiday', 'Functioning Day':'function_day'}, inplace=True)

In [ ]:
# Convert columns into categories
df.apply(lambda x: len(x.unique()))

date                  365
rented_bike_count    2166
hour                   24
temp                  546
humidity%              90
wind_speed             65
visibility           1789
dev_point_temp        556
solar_radiation       345
rainfall               61
snowfall               51
season                  4
holiday                 2
function_day            2
month                  12
weekday                 7
year                    2
dtype: int64

In [ ]:
# Clearly "season","holiday","function_day" are the categories
for col in ["season","holiday","function_day"]:
  df[col]=df[col].astype("category")